**NOTE:** 

Some additional modifications to the Multimodal-Toolkit source code ([Repository-Link](https://github.com/georgian-io/Multimodal-Toolkit)) with respect to loading the data have been performed (in particular the `load_data_from_folder` method). These changes were not tracked by version control!

In [1]:
! pip install multimodal-transformers

In [2]:
import json
import re
import sys
import pandas as pd
import numpy as np

## Preprocessing

In [3]:
import dataloader

In [4]:
train_df = dataloader.load_data(data_file = '../data/train.data.jsonl', label_file = '../data/train.label.json', perform_stemming = False)
dev_df = dataloader.load_data(data_file = '../data/dev.data.jsonl', label_file = '../data/dev.label.json', perform_stemming = False)
test_df = dataloader.load_data(data_file = '../data/test.data.jsonl', label_file = None, perform_stemming = False)

In [5]:
test_df['text'][0]

'people have been able to get out of sydney cafe during hostage situation fucking terrorists   people have been able to get out of sydney cafe during hostage situation her fingers look broken otages libres   people have been able to get out of sydney cafe definitely adding that tweet to my people have been able to get out of sydney cafe during hostage situation     cuba usa stop showing their faces  they are going to have enough trouble without forever being linked with this event i sure hope the hostage taker gets out  feet first and room temp people have been able to get out of sydney cafe during hostage situation dear mr  swat team guy  you may want to choose a long rifle for a more precise shot  this is not csi or miami vice radical islam again spreading terror  muslims are fanatical zealots p  isis syndeysiege anyone see the connection yet  how long r free and democratic nations going to allow terrorists and state sponsors of terror dictate'

## Multimodal BERT

In [6]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'

In [7]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """
    model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class MultimodalDataTrainingArguments:
    """
    Arguments pertaining to how we combine tabular features
    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
    column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
    })

    column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
    })

    categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
    numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
    task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

    mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
    combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
    mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
    numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
    use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
    mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
    gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

    def __post_init__(self):
        assert self.column_info != self.column_info_path
        if self.column_info is None and self.column_info_path:
            with open(self.column_info_path, 'r') as f:
                self.column_info = json.load(f)

In [8]:
text_cols = ['text']
cat_cols = ['question_mark', 'contains_url', 'contains_media', 'contains_profile_background_image', 'verified', 'geo_enabled', 'has_description']
numerical_cols = ['retweet_count', 'favorite_count', 'number_urls','statuses_count', 'listed_count', 'reputation_score_1', 'reputation_score_2', 'favourites_count','length_description','follow_tweets']

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'label',
    'label_list': [0, 1]
}


model_args = ModelArguments(
    model_name_or_path='bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    num_train_epochs=7,
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250,
    dataloader_drop_last=True
)

set_seed(training_args.seed)

In [9]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)

# Tokens automatically converted to lower_case
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir
)

Specified tokenizer:  bert-base-uncased


In [10]:
# Get Datasets
train_dataset, dev_dataset, test_dataset = load_data_from_folder(train_df, dev_df, test_df,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token
)

INFO:multimodal_transformers.data.data_utils:9 numerical columns
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
INFO:multimodal_transformers.data.data_utils:20 categorical columns
INFO:multimodal_transformers.data.data_utils:9 numerical columns
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
INFO:multimodal_transformers.data.load_data:Text

In [11]:
num_labels = len(np.unique(train_dataset.labels))
num_labels

2

In [12]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [13]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithTabular: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithTabular were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifi

In [14]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

def calc_classification_metrics(p: EvalPrediction):
    pred_labels = np.argmax(p.predictions, axis=1)
    pred_scores = softmax(p.predictions, axis=1)[:, 1]
    labels = p.label_ids
    if len(np.unique(labels)) == 2:  # binary classification
        roc_auc_pred_score = roc_auc_score(labels, pred_scores)
        precisions, recalls, thresholds = precision_recall_curve(labels,
                                                                pred_scores)
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        fscore[np.isnan(fscore)] = 0
        ix = np.argmax(fscore)
        threshold = thresholds[ix].item()
        pr_auc = auc(recalls, precisions)
        tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
        result = {'roc_auc': roc_auc_pred_score,
                'threshold': threshold,
                'pr_auc': pr_auc,
                'recall': recalls[ix].item(),
                'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
                'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
                }
    else:
        acc = (pred_labels == labels).mean()
        f1 = f1_score(y_true=labels, y_pred=pred_labels)
        result = {
          "acc": acc,
          "f1": f1,
          "acc_and_f1": (acc + f1) / 2,
          "mcc": matthews_corrcoef(labels, pred_labels)
        }

    return result

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=calc_classification_metrics
)

In [16]:
%%time
trainer.train()

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1160 [00:00<?, ?it/s]

{'loss': 0.7431385040283203, 'learning_rate': 4.9846059113300494e-05, 'epoch': 0.021551724137931036, 'step': 25}
{'loss': 0.598366470336914, 'learning_rate': 4.9692118226600986e-05, 'epoch': 0.04310344827586207, 'step': 50}
{'loss': 0.6772987365722656, 'learning_rate': 4.9538177339901484e-05, 'epoch': 0.06465517241379311, 'step': 75}
{'loss': 0.6353677368164062, 'learning_rate': 4.938423645320197e-05, 'epoch': 0.08620689655172414, 'step': 100}
{'loss': 0.6358514404296876, 'learning_rate': 4.923029556650247e-05, 'epoch': 0.10775862068965517, 'step': 125}
{'loss': 0.6349850463867187, 'learning_rate': 4.907635467980296e-05, 'epoch': 0.12931034482758622, 'step': 150}
{'loss': 0.6305630493164063, 'learning_rate': 4.892241379310345e-05, 'epoch': 0.15086206896551724, 'step': 175}
{'loss': 0.5048910522460938, 'learning_rate': 4.876847290640394e-05, 'epoch': 0.1724137931034483, 'step': 200}
{'loss': 0.5755682373046875, 'learning_rate': 4.8614532019704434e-05, 'epoch': 0.1939655172413793, 'step'

Evaluation:   0%|          | 0/72 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


...

Evaluation:   0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.545825412729755, 'eval_roc_auc': 0.8860904328646263, 'eval_threshold': 0.04786735773086548, 'eval_pr_auc': 0.7358323935900215, 'eval_recall': 0.8655913978494624, 'eval_precision': 0.706140350877193, 'eval_f1': 0.7777777777777778, 'eval_tn': 341, 'eval_fp': 49, 'eval_fn': 44, 'eval_tp': 142, 'epoch': 6.681034482758621, 'step': 7750}
{'loss': 0.476474609375, 'learning_rate': 2.124384236453202e-06, 'epoch': 6.702586206896552, 'step': 7775}
{'loss': 0.49689453125, 'learning_rate': 1.970443349753695e-06, 'epoch': 6.724137931034483, 'step': 7800}
{'loss': 0.438505859375, 'learning_rate': 1.816502463054187e-06, 'epoch': 6.745689655172414, 'step': 7825}
{'loss': 0.677421875, 'learning_rate': 1.66256157635468e-06, 'epoch': 6.767241379310345, 'step': 7850}
{'loss': 0.5513671875, 'learning_rate': 1.5086206896551726e-06, 'epoch': 6.788793103448276, 'step': 7875}
{'loss': 0.315, 'learning_rate': 1.3546798029556653e-06, 'epoch': 6.810344827586206, 'step': 7900}
{'loss': 0.33255859375

Evaluation:   0%|          | 0/72 [00:00<?, ?it/s]

{'eval_loss': 0.5048482806370076, 'eval_roc_auc': 0.8961538461538461, 'eval_threshold': 0.0877491757273674, 'eval_pr_auc': 0.7455978384892411, 'eval_recall': 0.8763440860215054, 'eval_precision': 0.7244444444444444, 'eval_f1': 0.7931873479318734, 'eval_tn': 337, 'eval_fp': 53, 'eval_fn': 38, 'eval_tp': 148, 'epoch': 6.896551724137931, 'step': 8000}
{'loss': 0.45513671875, 'learning_rate': 5.849753694581281e-07, 'epoch': 6.918103448275862, 'step': 8025}
{'loss': 0.2897265625, 'learning_rate': 4.3103448275862073e-07, 'epoch': 6.939655172413794, 'step': 8050}
{'loss': 0.34822265625, 'learning_rate': 2.7709359605911334e-07, 'epoch': 6.961206896551724, 'step': 8075}
{'loss': 0.6094140625, 'learning_rate': 1.2315270935960593e-07, 'epoch': 6.982758620689655, 'step': 8100}
CPU times: user 1h 35min 9s, sys: 30min 58s, total: 2h 6min 8s
Wall time: 2h 7min 54s


TrainOutput(global_step=8120, training_loss=0.5218131831126848)

In [17]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [18]:
%tensorboard --logdir ./logs/runs

In [19]:
trainer.save_model("./multimodal_bert/multimodal_bert_v37")

### Inference

In [20]:
# Need to reload trainer because drop_last = True was set during training 
# Otherwise the inference would not work on how test sets
pred_trained = Trainer(
    model=model
)

In [21]:
result = pred_trained.predict(test_dataset=test_dataset).predictions

Prediction:   0%|          | 0/73 [00:00<?, ?it/s]

In [22]:
from scipy.special import softmax

pred_labels = np.argmax(result, axis=1)
pred_scores = softmax(result, axis=1)[:, 1]

In [23]:
predicted_labels = [dataloader.convert_prediction(pred) for pred in pred_labels]

In [24]:
output = pd.DataFrame({'id':test_df.id,'target':predicted_labels})
output

,id,target
0,544382249178001408,rumour
1,525027317551079424,rumour
2,544273220128739329,rumour
3,499571799764770816,non-rumour
4,552844104418091008,non-rumour
...,...,...
576,553581227165642752,non-rumour
577,552816302780579840,non-rumour
578,580350000074457088,rumour
579,498584409055174656,non-rumour


In [25]:
submission = pd.Series(output.target.values,index=output.id).to_dict()
with open('test-output.json', 'w') as f:
    json.dump(submission, f)